In [ ]:
import os
import shutil
import hashlib
from twilio.rest import Client
from cryptography.fernet import Fernet
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QFileDialog, QMessageBox, QLineEdit
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Folder setup
UPLOADS_FOLDER = "uploads"
ENCRYPTED_FOLDER = "encrypted"
VERSIONS_FOLDER = "versions"
KEY_FILE = "key.key"

# Email and Twilio configuration
SENDER_EMAIL = "aasrithareddysg@gmail.com"
SENDER_PASSWORD = "zrfm fhnp xerg bjaq"
TWILIO_ACCOUNT_SID = "AC8d3ea9b5bfc25a6d98023ccc831a4528"
TWILIO_AUTH_TOKEN = "d96c9d7df5c7cf60bc6b121319c73956"
TWILIO_PHONE_NUMBER = "+916305356367"

try:
    twilio_client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
except Exception as e:
    print("Error initializing Twilio client:", e)


class FileSharingServer:
    def __init__(self):
        os.makedirs(UPLOADS_FOLDER, exist_ok=True)
        os.makedirs(ENCRYPTED_FOLDER, exist_ok=True)
        os.makedirs(VERSIONS_FOLDER, exist_ok=True)

        if not os.path.exists(KEY_FILE):
            key = Fernet.generate_key()
            with open(KEY_FILE, "wb") as key_file:
                key_file.write(key)
            print("[INFO] New encryption key generated.")
        with open(KEY_FILE, "rb") as key_file:
            self.key = key_file.read()
        self.cipher = Fernet(self.key)

    def encrypt_file(self, file_name, data):
        try:
            print(f"[DEBUG] Encrypting file: {file_name}")
            encrypted_data = self.cipher.encrypt(data)
            encrypted_file_path = os.path.join(ENCRYPTED_FOLDER, file_name)
            with open(encrypted_file_path, "wb") as file:
                file.write(encrypted_data)
            print(f"[SUCCESS] File encrypted and saved to: {encrypted_file_path}")
            return encrypted_file_path
        except Exception as e:
            print(f"[ERROR] Encryption failed: {e}")
            return None

    def decrypt_file(self, file_name):
        try:
            encrypted_file_path = os.path.join(ENCRYPTED_FOLDER, file_name)
            with open(encrypted_file_path, "rb") as file:
                encrypted_data = file.read()
            return self.cipher.decrypt(encrypted_data)
        except Exception as e:
            print(f"[ERROR] Decryption failed: {e}")
            return None

    def hash_file(self, data):
        hash_object = hashlib.sha256()
        hash_object.update(data)
        return hash_object.hexdigest()

    def upload_file(self, file_name, data, show_encryption_process=False):
        file_path = os.path.join(UPLOADS_FOLDER, file_name)
        if os.path.exists(file_path):
            return None, None
        with open(file_path, "wb") as file:
            file.write(data)
        if show_encryption_process:
            print("[DEBUG] Starting encryption process...")
        encrypted_file_path = self.encrypt_file(file_name, data)
        return file_name, encrypted_file_path if encrypted_file_path else (None, None)

    def download_file(self, file_name):
        file_path = os.path.join(UPLOADS_FOLDER, file_name)
        if os.path.exists(file_path):
            with open(file_path, "rb") as file:
                return file.read()
        return None

    def list_files(self):
        return os.listdir(UPLOADS_FOLDER)

    def create_version(self, file_name):
        file_path = os.path.join(UPLOADS_FOLDER, file_name)
        if os.path.exists(file_path):
            with open(file_path, "rb") as file:
                data = file.read()
            hash_value = self.hash_file(data)
            version_folder = os.path.join(VERSIONS_FOLDER, file_name)
            os.makedirs(version_folder, exist_ok=True)
            version_file_path = os.path.join(version_folder, hash_value)
            if not os.path.exists(version_file_path):
                shutil.copy(file_path, version_file_path)
                return True
        return False


class FileSharingClient:
    def __init__(self, server):
        self.server = server

    def send_email_notification(self, file_name, encrypted_file_path, receiver_email):
        subject = f"File '{file_name}' Upload Notification"
        body = f"File '{file_name}' has been uploaded securely. Encrypted copy is attached."

        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = receiver_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        try:
            with open(encrypted_file_path, 'rb') as attachment:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment.read())
                encoders.encode_base64(part)
                part.add_header('Content-Disposition', f'attachment; filename=encrypted_{file_name}')
                msg.attach(part)

            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, receiver_email, msg.as_string())
            server.quit()
            print("[INFO] Email notification sent.")
        except Exception as e:
            print(f"[ERROR] Failed to send email: {e}")

    def send_sms_notification(self, file_name, encrypted_path, receiver_email):
        message = f"File '{file_name}' has been securely uploaded and encrypted."
        try:
            twilio_client.messages.create(
                to=receiver_email,
                from_=TWILIO_PHONE_NUMBER,
                body=message
            )
            print("[INFO] SMS notification sent.")
        except Exception as e:
            print(f"[ERROR] Failed to send SMS: {e}")

    def upload_file(self, file_path, receiver_email):
        if not os.path.exists(file_path):
            QMessageBox.critical(None, "Error", f"File '{file_path}' not found.")
            return None

        show = QMessageBox.question(None, "Encryption?", "Show encryption steps?", QMessageBox.Yes | QMessageBox.No)
        show_encryption_process = show == QMessageBox.Yes

        file_name = os.path.basename(file_path)
        with open(file_path, "rb") as f:
            data = f.read()

        file_name, encrypted_file_path = self.server.upload_file(file_name, data, show_encryption_process)
        if file_name:
            self.send_email_notification(file_name, encrypted_file_path, receiver_email)
            self.send_sms_notification(file_name, encrypted_file_path, receiver_email)
            return file_name
        else:
            QMessageBox.warning(None, "Upload Failed", "File already exists.")
            return None

    def list_files(self):
        return self.server.list_files()


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Secure Document Sharing")

        self.output_label = QLabel("", self)
        self.output_label.setGeometry(50, 200, 500, 100)
        self.output_label.setAlignment(QtCore.Qt.AlignCenter)
        self.output_label.setStyleSheet("font-size: 16px;")
        self.output_label.setWordWrap(True)

        self.choice_button = QPushButton("Submit", self)
        self.choice_button.setGeometry(225, 150, 150, 30)
        self.choice_button.setStyleSheet("font-size: 14px;")
        self.choice_button.clicked.connect(self.handle_choice)

        self.receiver_email_label = QLabel("Receiver Email:", self)
        self.receiver_email_label.setGeometry(50, 100, 100, 30)
        self.receiver_email_label.setStyleSheet("font-size: 14px;")

        self.receiver_email_input = QLineEdit(self)
        self.receiver_email_input.setGeometry(160, 100, 200, 30)
        self.receiver_email_input.setStyleSheet("font-size: 14px;")

        self.previous_executions_label = QLabel("Previous Executions:", self)
        self.previous_executions_label.setGeometry(50, 350, 500, 30)
        self.previous_executions_label.setStyleSheet("font-size: 16px; font-weight: bold;")

        self.previous_executions_text = QLabel("", self)
        self.previous_executions_text.setGeometry(50, 400, 500, 150)
        self.previous_executions_text.setStyleSheet("font-size: 14px;")
        self.previous_executions_text.setWordWrap(True)

        self.previous_executions = []

    def handle_choice(self):
        user_choice, ok = QtWidgets.QInputDialog.getText(self, 'User Choice', "Enter 'U' to upload or 'Q' to quit:")
        if ok:
            choice = user_choice.strip().upper()
            client = FileSharingClient(server)
            receiver_email = self.receiver_email_input.text()

            if choice == 'U':
                file_path, _ = QFileDialog.getOpenFileName(self, "Select File to Upload")
                if file_path:
                    uploaded_file = client.upload_file(file_path, receiver_email)
                    if uploaded_file:
                        self.output_label.setText(f"File uploaded successfully: {uploaded_file}")
                        self.previous_executions.append(f"Uploaded: {uploaded_file}")
            elif choice == 'Q':
                self.close()
            else:
                QMessageBox.warning(self, "Invalid Choice", "Please enter 'U' or 'Q'.")
        self.update_previous_executions()

    def update_previous_executions(self):
        self.previous_executions_text.setText("\n".join(self.previous_executions))
        for log in self.previous_executions:
            print(log)


if __name__ == "__main__":
    app = QApplication([])
    server = FileSharingServer()
    window = MainWindow()
    window.setGeometry(200, 200, 800, 800)
    window.show()
    app.exec_()

[INFO] New encryption key generated.
[DEBUG] Starting encryption process...
[DEBUG] Encrypting file: sinwaveflowgraph.py
[SUCCESS] File encrypted and saved to: encrypted\sinwaveflowgraph.py
[INFO] Email notification sent.
[ERROR] Failed to send SMS: HTTP 400 error: Unable to create record: Invalid 'To' Phone Number: vaishnavivemula09@gmailXXXX
Uploaded: sinwaveflowgraph.py


In [3]:
# decrypt_file.py

import os
from cryptography.fernet import Fernet

# Paths
ENCRYPTED_FOLDER = r"C:\Users\LENOVO\Downloads\Secure_Document_Sharing\Secure_Document_Sharing\encrypted"
DECRYPTED_FOLDER = r"C:\Users\LENOVO\Desktop\decrypt"
KEY_FILE = r"C:\Users\LENOVO\Downloads\Secure_Document_Sharing\Secure_Document_Sharing\key.key"

# Ensure decrypted folder exists
os.makedirs(DECRYPTED_FOLDER, exist_ok=True)

# Load key
if not os.path.exists(KEY_FILE):
    print("[ERROR] Key file 'key.key' not found.")
    exit(1)

with open(KEY_FILE, "rb") as key_file:
    key = key_file.read()
    print("[INFO] Key loaded.")

cipher = Fernet(key)

# Get encrypted file name
file_name = input("Enter the name of the file to decrypt (inside 'encrypted/' folder): ").strip()
encrypted_file_path = os.path.join(ENCRYPTED_FOLDER, file_name)
decrypted_file_path = os.path.join(DECRYPTED_FOLDER, file_name)

if not os.path.exists(encrypted_file_path):
    print(f"[ERROR] Encrypted file '{encrypted_file_path}' not found.")
    exit(1)

with open(encrypted_file_path, "rb") as f:
    encrypted_data = f.read()
    print(f"[INFO] Encrypted file loaded. Size: {len(encrypted_data)} bytes")

try:
    decrypted_data = cipher.decrypt(encrypted_data)
    with open(decrypted_file_path, "wb") as f:
        f.write(decrypted_data)
    print(f"[SUCCESS] File decrypted and saved to '{decrypted_file_path}'.")
except Exception as e:
    print(f"[ERROR] Decryption failed: {type(e).__name__} - {e}")

[INFO] Key loaded.


[ERROR] Encrypted file 'C:\Users\LENOVO\Downloads\Secure_Document_Sharing\Secure_Document_Sharing\encrypted\Desktop_based_Smart_Voice_Assistant_using_Python_Langauage_Integrated_with_Arduino.pdf' not found.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\LENOVO\\Downloads\\Secure_Document_Sharing\\Secure_Document_Sharing\\encrypted\\Desktop_based_Smart_Voice_Assistant_using_Python_Langauage_Integrated_with_Arduino.pdf'